In [1]:
import pandas as pd
import numpy as np
#from polygon import RESTClient
import datetime
import json
import requests
import os
import math
from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

In [2]:
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
#import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

#Libraries for Deep Learning Models
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasRegressor

#Libraries for Statistical Models
import statsmodels.api as sm

# Time series Models
from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.statespace.sarimax import SARIMAX

# Error Metrics
from sklearn.metrics import mean_squared_error

# Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression


#Plotting 
from pandas.plotting import scatter_matrix
from statsmodels.graphics.tsaplots import plot_acf


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def financials(ticker):
    financials = "https://api.polygon.io/v2/reference/financials/"+ticker+"?limit=120&type=Q&apiKey=XrTJjjYQpsWD5i9Gn1vBE1LEu1J1R5gJ"
    response_data = requests.get(financials).json()
    df = pd.DataFrame(response_data['results'])
    #df = df[["calendarDate", "assetsCurrent", 'cashAndEquivalentsUSD', "sharePriceAdjustedClose", 'investmentsCurrent', 'assetsCurrent', 'debtToEquityRatio', 'debtUSD', 'dividendYield', 'earningsBeforeInterestTaxesDepreciationAmortizationUSD', 'EBITDAMargin', 'shareholdersEquityUSD', 'enterpriseValue']]
    return df

In [5]:
tickers = ['HD']#, 'LOW', 'CAT', 'KR', 'WMT']
test_df = pd.DataFrame()
for ticker in tickers:
    print(ticker)
    data = financials(ticker)
    #data = data.set_index('calendarDate')
    data = data.drop(['ticker', 'period', 'reportPeriod', 'updated', 'dateKey' ], axis=1)
    data = data.add_prefix(f'{ticker}_')
    test_df = pd.concat([test_df, data], axis=1, join='outer')



HD


In [6]:
test_df = test_df.rename(columns={'HD_calendarDate': 'calendarDate'})

#for ticker in tickers[1:]:
#    test_df2 = test_df.drop([f'{ticker}_calendarDate'], axis=1)

In [7]:
test_df=test_df.set_index('calendarDate')


In [8]:
test_df['HD_enterpriseValueOverEBITDA']

calendarDate
2019-12-31    12.712
2019-09-30    15.147
2019-06-30    15.102
2019-03-31    13.474
2018-12-31    13.417
               ...  
1996-12-31    16.238
1996-09-30       NaN
1996-06-30       NaN
1996-06-30       NaN
1996-03-31       NaN
Name: HD_enterpriseValueOverEBITDA, Length: 98, dtype: float64

In [9]:
#ev_df = pd.DataFrame()
#ev_ebitda_df = pd.DataFrame()

#for ticker in tickers:
#    ev_df = pd.concat([ev_df,test_df[f'{ticker}_enterpriseValue']], axis=1, join='outer')
#    ev_ebitda_df = pd.concat([ev_ebitda_df,test_df[f'{ticker}_enterpriseValueOverEBITDA']], axis=1, join='outer')



In [10]:
#ev_df

In [11]:
#ev_ebitda_df
#ev_data = test_df.drop(['HD_enterpriseValueOverEBITDA'], axis =1)

In [12]:
#ev_data.dtypes

In [13]:
return_period = 4

Y = np.log(test_df.loc[:,'HD_enterpriseValueOverEBITDA']).diff(return_period).shift(-return_period)
Y.name = Y.name[-1]+'_pred'
#Y = Y.rename(columns={'HD_enterpriseValueOverEBITDA': 'HD_enterpriseValueOverEBITDA_pred'})

Y



calendarDate
2019-12-31    0.053976
2019-09-30   -0.207959
2019-06-30   -0.037374
2019-03-31    0.048953
2018-12-31    0.007352
                ...   
1996-12-31         NaN
1996-09-30         NaN
1996-06-30         NaN
1996-06-30         NaN
1996-03-31         NaN
Name: A_pred, Length: 98, dtype: float64

In [17]:
#col = test_df.columns.values.tolist()
#col.remove("HD_enterpriseValueOverEBITDA")
#col
test_df3 = test_df.drop(columns=['HD_enterpriseValueOverEBITDA'])

In [18]:
#for ticker in tickers:
X1 = np.log(test_df3).diff(return_period)
#test_df.loc[:, col]
#X1.columns = X1.columns.droplevel()
#X2 = np.log(ccy_data).diff(return_period)
#X3 = np.log(idx_data).diff(return_period)

X4 = pd.concat([np.log(test_df.loc[:,'HD_enterpriseValueOverEBITDA' ]).diff(i) for i in [return_period, return_period*2, return_period*4, return_period*8]], axis=1).dropna()
X4.columns = [f'{ticker}_DT', f'{ticker}_2DT', f'{ticker}_4DT', f'{ticker}_8DT']

X = pd.concat([X1, X4], axis=0, join='outer')


dataset = pd.concat([Y, X], axis=0, join='outer')#.iloc[::return_period, :]
#Y = dataset.loc[:, Y.columns]
#X = dataset.loc[:, X.columns]
dataset

,0,HD_2DT,HD_4DT,HD_8DT,HD_DT,HD_EBITDAMargin,HD_accumulatedOtherComprehensiveIncome,HD_accumulatedRetainedEarningsDeficit,HD_assets,HD_assetsCurrent,...,HD_tangibleAssetValue,HD_tangibleAssetsBookValuePerShare,HD_taxAssets,HD_taxLiabilities,HD_totalLiabilities,HD_tradeAndNonTradePayables,HD_tradeAndNonTradeReceivables,HD_weightedAverageShares,HD_weightedAverageSharesDiluted,HD_workingCapital
calendarDate,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0.053976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30,-0.207959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-30,-0.037374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-31,0.048953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31,0.007352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997-12-31,NaN,-0.295897,0.224699,1.026499,-0.266040,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-09-30,NaN,-0.212755,-0.070641,0.864962,-0.366933,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-06-30,NaN,-0.419320,-0.325196,0.634149,-0.322459,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
date_df = pd.DataFrame()
for ticker in tickers:
    date_df = pd.concat([date_df,test_df[f'{ticker}_calendarDate']], axis=1, join='outer')

date_df


,HD_calendarDate,LOW_calendarDate,CAT_calendarDate,KR_calendarDate,WMT_calendarDate
0,2019-12-31,2019-12-31,2019-12-31,2020-03-31,2019-12-31
1,2019-09-30,2019-09-30,2019-09-30,2019-12-31,2019-09-30
2,2019-06-30,2019-06-30,2019-06-30,2019-09-30,2019-06-30
3,2019-03-31,2019-03-31,2019-03-31,2019-06-30,2019-03-31
4,2018-12-31,2018-12-31,2018-12-31,2019-03-31,2018-12-31
5,2018-09-30,2018-09-30,2018-09-30,2018-12-31,2018-09-30
6,2018-06-30,2018-06-30,2018-06-30,2018-09-30,2018-06-30
7,2018-03-31,2018-03-31,2018-03-31,2018-06-30,2018-03-31
8,2017-12-31,2017-12-31,2017-12-31,2018-03-31,2017-12-31
9,2017-09-30,2017-09-30,2017-09-30,2017-12-31,2017-09-30


In [ ]:
balance_sheet_trend_analysis = test_df.copy()
balance_sheet_trend_analysis = balance_sheet_trend_analysis.set_index('ticker').T
balance_sheet_trend_analysis

ticker,HD,HD,HD,HD,HD,HD,HD,HD,HD,HD,...,WMT,WMT,WMT,WMT,WMT,WMT,WMT,WMT,WMT,WMT
period,Q,Q,Q,Q,Q,Q,Q,Q,Q,Q,...,Q,Q,Q,Q,Q,Q,Q,Q,Q,Q
cashAndEquivalentsUSD,2133000000,2193000000,2547000000,1882000000,1778000000,1764000000,3490000000,3599000000,3595000000,3549000000,...,8643000000,7935000000,8117000000,6550000000,7063000000,8102000000,9400000000,7395000000,10616000000,10195000000
investmentsCurrent,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
assetsCurrent,19810000000,21174000000,20699000000,20553000000,18529000000,19809000000,20802000000,21214000000,18933000000,19682000000,...,63431000000,56259000000,57276000000,54975000000,59271000000,55414000000,55958000000,51893000000,59568000000,53045000000
debtToEquityRatio,-17.443,-49.345,-45.836,-25.039,-24.431,33.242,22.012,26.653,29.625,16.705,...,1.722,1.721,1.786,1.65,1.841,1.778,1.816,1.596,1.814,1.702
debtUSD,37377000000,35051000000,34473000000,34198000000,29202000000,25784000000,25498000000,25793000000,27028000000,25589000000,...,57457000000,54623000000,54626000000,53427000000,59236000000,57078000000,55666000000,49864000000,56794000000,49233000000
dividendYield,0.031,0.023,0.022,0.023,0.023,0.024,0.019,0.02,0.021,0.019,...,0.022,0.021,0.023,0.024,0.025,0.027,0.025,0.024,0.022,0.023
earningsBeforeInterestTaxesDepreciationAmortizationUSD,3998000000,4541000000,5456000000,4144000000,3927000000,4411000000,5431000000,3913000000,3718000000,4198000000,...,8058000000,8730000000,8379000000,10220000000,7818000000,8311000000,7746000000,10928000000,7397000000,7980000000
EBITDAMargin,0.155,0.167,0.177,0.157,0.148,0.168,0.178,0.157,0.156,0.168,...,0.071,0.076,0.074,0.083,0.071,0.076,0.074,0.094,0.073,0.077
shareholdersEquityUSD,-3116000000,-1082000000,-1160000000,-2143000000,-1878000000,1320000000,2009000000,1687000000,1454000000,2543000000,...,73891000000,70278000000,68964000000,71315000000,67163000000,67941000000,65006000000,68542000000,65525000000,64646000000
